# Segmenting and Clustering Neighborhoods in Toronto
- This notebook is for the ibm coursera applied data science week 3 project
- It is built using a docker image  and jupyterLabs
- the design for this notebook is based on the week 3 Segmentation methods labs

Objective: "For this assignment, you will be required to explore and cluster the neighborhoods in Toronto."



## Author Franklin Bettencourt
## 2020-APR-12

In [65]:
import numpy as np # library to handle data in a vectorized manner
#!conda install -c conda-forge lxml --yes

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Get DataFrame From Wiki Page into Pandas Data Frame 

Could not get the webscraping to work so I had to copy page to excel to then import the table. Since this was not the focus of this project I figured it was ok to do so. 

In [66]:
# Load in WikiTable Data 

df = pd.read_csv("torontoWikiTableRaw.csv")
print(df.columns)
print(df.shape)
df.head()

Index(['Postal code', 'Borough', 'Neighborhood'], dtype='object')
(180, 3)


Postal code            Borough                 Neighborhood
0        M1A       Not assigned                           NaN
1        M2A       Not assigned                           NaN
2        M3A         North York                    Parkwoods 
3        M4A         North York             Victoria Village 
4        M5A   Downtown Toronto   Regent Park / Harbourfront

In [67]:
#help(pd.DataFrame.loc)

In [68]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
# done 
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#we need to drop all rows with borough==NotAssigned
notAs=df.loc[0,'Borough']
iNotAs= [df.loc[:,'Borough'] != notAs]
#type(iNotAs[0])

dfAssnd = df.loc[iNotAs[0],:]
dfAssnd.head(10)

Postal code            Borough  \
2         M3A         North York    
3         M4A         North York    
4         M5A   Downtown Toronto    
5         M6A         North York    
6         M7A   Downtown Toronto    
8         M9A          Etobicoke    
9         M1B        Scarborough    
11        M3B         North York    
12        M4B          East York    
13        M5B   Downtown Toronto    

                                     Neighborhood  
2                                      Parkwoods   
3                               Victoria Village   
4                     Regent Park / Harbourfront   
5              Lawrence Manor / Lawrence Heights   
6   Queen's Park / Ontario Provincial Government   
8                               Islington Avenue   
9                                Malvern / Rouge   
11                                     Don Mills   
12              Parkview Hill / Woodbine Gardens   
13                      Garden District, Ryerson

In [69]:
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is 
#listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods 
#separated with a comma as shown in row 11 in the above table.
#dfAssnd.loc[dfAssnd.loc[:,'Postal code']=="M5A ",:]

## Answer - It appread that there was a 1:1 corrlecation between Borough = "Not assigned" and Neighborhood = "NaN"

## scratch work 
#isNan=df.loc[0,'Neighborhood']
#print(isNan)
##iNotNan= [df.loc[:,'Neighborhood'] != isNan]
#iNotNan= [df.loc[:,'Neighborhood'] != "NaN"]
##print(iNotNan)
#dfNotNan=df.loc[iNotNan[0],:]
#dfNotNan.head(15)
##         #dfAssnd = df.loc[iNotAs[0],:]
##dfAssnd.head(15)

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
#np.unique(dfAssnd[['Neighborhood']].values)

In [70]:
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
dfAssnd.shape

(103, 3)

# Get Long and Lats into Pandas Data from for each Neighborhood

In [71]:
!pip install geocoder

In [72]:
# This code ran endlessly so I am not sure how to fix this
# as a result I am going to use the provided csv file containing 
# the Lats and Longs for the neighbs 

#import geocoder # import geocoder

## initialize your variable to None
#lat_lng_coords = None
#postal_code = "M5G "
## loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [73]:
## reading in CSV 
latsLongs = pd.read_csv("Geospatial_Coordinates.csv")
latsLongs.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [74]:
## merge data frames by Postal Code 
#help(pd.DataFrame.join)
#dfAssnd.columns
import re

#dfAssnd.loc[:,'Postal code']
#dmy=re.sub(" ", "", dfAssnd.loc[:,'Postal code'].values)

#dfAssnd.loc[;,'Postal code']=
dfAssnd.loc[:,'Postal code']=dfAssnd.loc[:,'Postal code'].replace(" ", "", regex=True)
#dfAll = dfAs.snd.merge(latsLongs,
#                      left_on="Postal code", 
#                      right_on="Postal Code",
#                     left_index=False, 
#                     right_index=False)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [75]:
dfAssnd.loc[:,'Postal code'].values #check if data was cleaned 

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

In [76]:
dfAll = dfAssnd.merge(latsLongs,
                      left_on="Postal code", 
                      right_on="Postal Code",
                     left_index=False, 
                     right_index=False)
dfAll.head()

Postal code            Borough  \
0         M3A        North York    
1         M4A        North York    
2         M5A  Downtown Toronto    
3         M6A        North York    
4         M7A  Downtown Toronto    

                                    Neighborhood Postal Code   Latitude  \
0                                     Parkwoods          M3A  43.753259   
1                              Victoria Village          M4A  43.725882   
2                    Regent Park / Harbourfront          M5A  43.654260   
3             Lawrence Manor / Lawrence Heights          M6A  43.718518   
4  Queen's Park / Ontario Provincial Government          M7A  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494

# Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

#### Here I begin the process of mimicing the analysis we did for the NYC example project

In [77]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [78]:
CLIENT_ID = 'IZ5PSQQLYNDHZKX1UL3NUTTINWZ05EIXJS4CM44H1EI3F3TY' # your Foursquare ID
CLIENT_SECRET = '2I0TXBSSKBXJTUVNOXLNCYEWJMXXZ3XLSAEQ5N5BSXYQHPOX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

radius = 500
LIMIT  = 120

Your credentails:
CLIENT_ID: IZ5PSQQLYNDHZKX1UL3NUTTINWZ05EIXJS4CM44H1EI3F3TY
CLIENT_SECRET:2I0TXBSSKBXJTUVNOXLNCYEWJMXXZ3XLSAEQ5N5BSXYQHPOX


In [79]:
dfAll.loc[0,:]
lat=dfAll.loc[0,"Latitude"]
lng=dfAll.loc[0,"Longitude"]
print(lat," ", lng)

43.7532586   -79.3296565


In [80]:
# test with first neigh
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
# make the GET request
results = requests.get(url).json()["response"]['groups'][0]['items']
results

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
   'name': 'Brookbanks Park',
   'location': {'address': 'Toronto',
    'lat': 43.751976046055574,
    'lng': -79.33214044722958,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.751976046055574,
      'lng': -79.33214044722958}],
    'distance': 245,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d163941735',
     'name': 'Park',
     'pluralName': 'Parks',
     'shortName': 'Park',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'},
 {'reasons': {'count': 0,
   'items': [{'

In [81]:
toronto_venues=getNearbyVenues(names=dfAll.loc[:,'Neighborhood'],
                                   latitudes=dfAll.loc[:,'Latitude'],
                                   longitudes=dfAll.loc[:,'Longitude'])  

Parkwoods 
Victoria Village 
Regent Park / Harbourfront 
Lawrence Manor / Lawrence Heights 
Queen's Park / Ontario Provincial Government 
Islington Avenue 
Malvern / Rouge 
Don Mills 
Parkview Hill / Woodbine Gardens 
Garden District, Ryerson 
Glencairn 
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale 
Rouge Hill / Port Union / Highland Creek 
Don Mills 
Woodbine Heights 
St. James Town 
Humewood-Cedarvale 
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood 
Guildwood / Morningside / West Hill 
The Beaches 
Berczy Park 
Caledonia-Fairbanks 
Woburn 
Leaside 
Central Bay Street 
Christie 
Cedarbrae 
Hillcrest Village 
Bathurst Manor / Wilson Heights / Downsview North 
Thorncliffe Park 
Richmond / Adelaide / King 
Dufferin / Dovercourt Village 
Scarborough Village 
Fairview / Henry Farm / Oriole 
Northwood Park / York University 
East Toronto 
Harbourfront East / Union Station / Toronto Islands 
Little Portugal / Trinity 
Kennedy Park / Ionview / E

In [82]:
print(toronto_venues.shape)
toronto_venues.head()

(2143, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods               43.753259              -79.329656   
1         Parkwoods               43.753259              -79.329656   
2  Victoria Village               43.725882              -79.315572   
3  Victoria Village               43.725882              -79.315572   
4  Victoria Village               43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1           Variety Store       43.751974       -79.333114   
2  Victoria Village Arena       43.723481       -79.315635   
3             Tim Hortons       43.725517       -79.313103   
4               Portugril       43.725819       -79.312785   

          Venue Category  
0                   Park  
1      Food & Drink Shop  
2           Hockey Arena  
3            Coffee Shop  
4  Portuguese Restaurant

How many Venues were returned for each Neighb?

In [83]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Agincourt                                                               4   
Alderwood / Long Branch                                                10   
Bathurst Manor / Wilson Heights / Downsview North                      19   
Bayview Village                                                         4   
Bedford Park / Lawrence Manor East                                     23   
Berczy Park                                                            55   
Birch Cliff / Cliffside West                                            4   
Brockton / Parkdale Village / Exhibition Place                         24   
Business reply mail Processing CentrE                                  19   
CN Tower / King and Spadina / Railway Lands / H...                     17   
Caledonia-Fairbanks                                                     4   
Canada Post Gateway Processing Centre                                  13   
Cedarbrae                                                               9   
Central Bay Street                                                     65   
Christie                                                               17   
Church and Wellesley                                                   74   
Clarks Corners / Tam O'Shanter / Sullivan                              12   
Cliffside / Cliffcrest / Scarborough Village West                       2   
Commerce Court / Victoria Hotel                                       100   
Davisville                                                             34   
Davisville North                                                       11   
Del Ray / Mount Dennis / Keelsdale and Silverth...                      3   
Don Mills                                                              28   
Dorset Park / Wexford Heights / Scarborough Tow...                      7   
Downsview                                                              15   
Dufferin / Dovercourt Village                                          15   
East Toronto                                                            3   
Eringate / Bloordale Gardens / Old Burnhamthorp...                      7   
Fairview / Henry Farm / Oriole                                         63   
First Canadian Place / Underground city                               100   
Forest Hill North & West                                                4   
Garden District, Ryerson                                              100   
Glencairn                                                               4   
Golden Mile / Clairlea / Oakridge                                      10   
Guildwood / Morningside / West Hill                                     7   
Harbourfront East / Union Station / Toronto Isl...                    100   
High Park / The Junction South                                         25   
Hillcrest Village                                                       4   
Humber Summit                                                           2   
Humberlea / Emery                                                       2   
Humewood-Cedarvale                                                      4   
India Bazaar / The Beaches West                                        20   
Kennedy Park / Ionview / East Birchmount Park                           7   
Kensington Market / Chinatown / Grange Park                            62   
Kingsview Village / St. Phillips / Martin Grove...                      4   
Lawrence Manor / Lawrence Heights                                      14   
Lawrence Park                                                           3   
Leaside                                                                33   
Little Portugal / Trinity                                              43   
Malvern / Rouge                                                         2   
Milliken / Agincourt North / Steeles East / L'A.

#### Let's find out how many unique categories can be curated from all the returned venues

In [84]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 264 uniques categories.


## Here we will begin to utilize the same analysis that was done in Section 3 of the Neighborhood clustering assignment from the previous week. 

In [85]:
# one hot encoding
tnto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tnto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tnto_onehot.columns[-1]] + list(tnto_onehot.columns[:-1])
tnto_onehot = tnto_onehot[fixed_columns]

tnto_onehot.head()

Yoga Studio  Accessories Store  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Arts & Crafts Store  Asian Restaurant  \
0             0         0            0                    0                 0   
1             0         0            0                    0                 0   
2             0         0            0                    0                 0   
3             0         0            0                    0                 0   
4             0         0            0                    0                 0   

   Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  \
0                   0              0          0           0           0   
1                   0              0          0           0           0   
2                   0              0          0           0           0   
3                   0              0          0           0           0   
4                   0              0          0           0           0   

   Bakery  Bank  Bar  Baseball Field  Baseball Stadium  Basketball Court  \
0       0     0    0               0                 0                 0   
1       0     0    0               0                 0                 0   
2       0     0    0               0                 0                 0   
3       0     0    0               0                 0                 0   
4       0     0    0               0                 0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bike Shop  Bistro  Board Shop  Boat or Ferry  \
0                   0          0       0           0              0   
1                   0          0       0           0              0   
2                   0          0       0           0              0   
3                   0          0       0           0              0   
4                   0          0       0           0              0   

   Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  \
0          0         0                     0               0        0   
1          0         0                     0               0        0   
2          0         0                     0               0        0   
3          0         0                     0               0        0   
4          0         0                     0               0        0   

   Bridal Shop  Bubble Tea Shop  Building  Burger Joint  Burrito Place  \
0            0                0         0             0              0   
1            0                0         0             0              0   
2            0                0         0             0              0   
3            0                0         0             0              0   
4

### lets look at the shape of our summary data 

In [86]:
tnto_onehot.shape

(2143, 264)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [87]:
tnto_grouped = tnto_onehot.groupby('Neighborhood').mean().reset_index()
tnto_grouped.head()

Neighborhood  Yoga Studio  \
0                                         Agincourt           0.0   
1                           Alderwood / Long Branch           0.0   
2  Bathurst Manor / Wilson Heights / Downsview No...          0.0   
3                                   Bayview Village           0.0   
4                Bedford Park / Lawrence Manor East           0.0   

   Accessories Store  Airport  Airport Food Court  Airport Gate  \
0                0.0      0.0                 0.0           0.0   
1                0.0      0.0                 0.0           0.0   
2                0.0      0.0                 0.0           0.0   
3                0.0      0.0                 0.0           0.0   
4                0.0      0.0                 0.0           0.0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0             0.0              0.0               0.0             0.000000   
1             0.0              0.0               0.0             0.000000   
2             0.0              0.0               0.0             0.000000   
3             0.0              0.0               0.0             0.000000   
4             0.0              0.0               0.0             0.043478   

   Antique Shop  Aquarium  Art Gallery  Arts & Crafts Store  Asian Restaurant  \
0           0.0       0.0          0.0                  0.0               0.0   
1           0.0       0.0          0.0                  0.0               0.0   
2           0.0       0.0          0.0                  0.0               0.0   
3           0.0       0.0          0.0                  0.0               0.0   
4           0.0       0.0          0.0                  0.0               0.0   

   Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  \
0                 0.0            0.0        0.0         0.0         0.0   
1                 0.1            0.0        0.0         0.0         0.0   
2                 0.0            0.0        0.0         0.0         0.0   
3                 0.0            0.0        0.0         0.0         0.0   
4                 0.0            0.0        0.0         0.0         0.0   

   Bakery      Bank  Bar  Baseball Field  Baseball Stadium  Basketball Court  \
0     0.0  0.000000  0.0             0.0               0.0               0.0   
1     0.0  0.000000  0.0             0.0               0.0               0.0   
2     0.0  0.105263  0.0             0.0               0.0               0.0   
3     0.0  0.250000  0.0             0.0               0.0               0.0   
4     0.0  0.000000  0.0             0.0               0.0               0.0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                 0.0    0.0              0.0       0.0         0.0   
1                 0.0    0.0              0.0       0.0         0.0   
2                 0.0    0.0              0.0       0.0         0.0   
3                 0.0    0.0              0.0       0.0         0.0   
4                 0.0    0.0              0.0       0.0         0.0   

   Belgian Restaurant  Bike Shop  Bistro  Board Shop  Boat or Ferry  \
0                 0.0        0.0     0.0         0.0            0.0   
1                 0.0        0.0     0.0         0.0            0.0   
2                 0.0        0.0     0.0         0.0            0.0   
3                 0.0        0.0     0.0         0.0            0.0   
4                 0.0        0.0     0.0         0.0            0.0   

   Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  \
0        0.0       0.0                   0.0            0.25      0.0   
1        0.0       0.0                   0.0            0.00      0.0   
2        0.0       0.0                   0.0            0.00      0.0   
3        0.0       0.0                   0.0            0.00      0.0   
4        0.0       0.0                   0.0            0.00      0.0   

   Bridal Shop  Bubble Tea Shop  Building  Burg

In [88]:
tnto_grouped.shape

(95, 264)

#### Let's print first 5 neighborhoods along with the top 5 most common venues in each

In [89]:
num_top_venues = 5

for hood in tnto_grouped['Neighborhood'][1:5]:
    print("----"+hood+"----")
    temp = tnto_grouped[tnto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood / Long Branch ----
                venue  freq
0         Pizza Place   0.2
1  Athletics & Sports   0.1
2            Pharmacy   0.1
3         Coffee Shop   0.1
4                Pool   0.1


----Bathurst Manor / Wilson Heights / Downsview North ----
         venue  freq
0  Coffee Shop  0.11
1         Bank  0.11
2  Pizza Place  0.05
3  Supermarket  0.05
4   Restaurant  0.05


----Bayview Village ----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4                Motel  0.00


----Bedford Park / Lawrence Manor East ----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.09
2          Restaurant  0.09
3      Sandwich Place  0.09
4        Liquor Store  0.04




#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [90]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [91]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tnto_grouped['Neighborhood']

for ind in np.arange(tnto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tnto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood  \
0                                         Agincourt    
1                           Alderwood / Long Branch    
2  Bathurst Manor / Wilson Heights / Downsview No...   
3                                   Bayview Village    
4                Bedford Park / Lawrence Manor East    

       1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0  Latin American Restaurant          Skating Rink                Lounge   
1                Pizza Place          Skating Rink                  Pool   
2                Coffee Shop                  Bank           Pizza Place   
3         Chinese Restaurant                  Café                  Bank   
4                Coffee Shop            Restaurant        Sandwich Place   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0        Breakfast Spot          Dessert Shop     Electronics Store   
1           Coffee Shop                   Gym                   Pub   
2         Deli / Bodega      Sushi Restaurant        Ice Cream Shop   
3   Japanese Restaurant         Women's Store                 Diner   
4    Italian Restaurant         Grocery Store      Sushi Restaurant   

         7th Most Common Venue      8th Most Common Venue  \
0  Eastern European Restaurant        Dumpling Restaurant   
1                     Pharmacy         Athletics & Sports   
2                Shopping Mall  Middle Eastern Restaurant   
3               Discount Store        Distribution Center   
4      Comfort Food Restaurant                   Pharmacy   

  9th Most Common Venue 10th Most Common Venue  
0             Drugstore             Donut Shop  
1        Sandwich Place       Department Store  
2            Restaurant                  Diner  
3               Dog Run       Doner Restaurant  
4           Pizza Place                   Café

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [121]:
# set number of clusters
kclusters = 10

tnto_grouped_clustering = tnto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=10).fit(tnto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 9, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 8, 3, 1, 1, 1, 1,
       1, 1, 4, 1, 1, 7, 2, 1, 2, 1, 4, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 2,
       1, 7, 1, 4, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 5, 1,
       9, 1, 1, 1, 1, 6, 9], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [122]:
# add clustering labels
if(("Cluster Labels" != neighborhoods_venues_sorted.columns[0])):
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tnto_merged = dfAll # dfAll was data frame that contained all data from foursquare query

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tnto_merged = tnto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tnto_merged.head() # check the last columns!

Postal code            Borough  \
0         M3A        North York    
1         M4A        North York    
2         M5A  Downtown Toronto    
3         M6A        North York    
4         M7A  Downtown Toronto    

                                    Neighborhood Postal Code   Latitude  \
0                                     Parkwoods          M3A  43.753259   
1                              Victoria Village          M4A  43.725882   
2                    Regent Park / Harbourfront          M5A  43.654260   
3             Lawrence Manor / Lawrence Heights          M6A  43.718518   
4  Queen's Park / Ontario Provincial Government          M7A  43.662301   

   Longitude  Cluster Labels 1st Most Common Venue   2nd Most Common Venue  \
0 -79.329656             4.0                  Park       Food & Drink Shop   
1 -79.315572             0.0           Coffee Shop             Pizza Place   
2 -79.360636             0.0           Coffee Shop                  Bakery   
3 -79.464763             0.0        Clothing Store  Furniture / Home Store   
4 -79.389494             0.0           Coffee Shop                   Diner   

  3rd Most Common Venue 4th Most Common Venue        5th Most Common Venue  \
0         Deli / Bodega     Electronics Store  Eastern European Restaurant   
1          Intersection          Hockey Arena        Portuguese Restaurant   
2                  Park                   Pub                         Café   
3           Event Space     Accessories Store                     Boutique   
4           Yoga Studio              Creperie                     Beer Bar   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0    Dumpling Restaurant             Drugstore            Donut Shop   
1          Women's Store   Distribution Center          Dessert Shop   
2         Breakfast Spot               Theater            Restaurant   
3  Vietnamese Restaurant             Gift Shop    Miscellaneous Shop   
4         Sandwich Place          Burger Joint         Burrito Place   

  9th Most Common Venue 10th Most Common Venue  
0      Doner Restaurant                Dog Run  
1    Dim Sum Restaurant                  Diner  
2    Mexican Restaurant         Chocolate Shop  
3           Coffee Shop    Distribution Center  
4                  Café     College Auditorium

Finally, let's visualize the resulting clusters

In [123]:
#help(pd.DataFrame.dropna)
# some of the neighbs get a cluster value of nan I am nto sure why ... so I have to filter them out here

print(np.unique(tnto_merged.loc[:,"Cluster Labels"]))

tnto_merged.dropna(inplace=True)

print(np.unique(tnto_merged.loc[:,"Cluster Labels"]))

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. nan nan nan]
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [124]:
# create map

#map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
map_clusters = folium.Map(location=[ dfAll.loc[0,'Latitude'], dfAll.loc[0,'Longitude'] ], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tnto_merged['Latitude'], tnto_merged['Longitude'], tnto_merged['Neighborhood'], tnto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Display Summary DataFrames of Each Cluster Created By kmeans

In [129]:
from IPython.display import display
for i in range(0,kclusters):
    print("Cluster ", i , " Summary")
    #tnto_merged.loc[tnto_merged['Cluster Labels'] == 3, tnto_merged.columns[[1] + list(range(5, tnto_merged.shape[1]))]]
    summarydf = tnto_merged.loc[tnto_merged['Cluster Labels'] == i, tnto_merged.columns[[1] + list(range(5, tnto_merged.shape[1]))]]
    display(summarydf.head(5))
    print("           ")

Cluster  0  Summary


Borough  Longitude  Cluster Labels 1st Most Common Venue  \
1        North York  -79.315572             0.0           Coffee Shop   
2  Downtown Toronto  -79.360636             0.0           Coffee Shop   
3        North York  -79.464763             0.0        Clothing Store   
4  Downtown Toronto  -79.389494             0.0           Coffee Shop   
7        North York  -79.352188             0.0           Coffee Shop   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
1             Pizza Place          Intersection          Hockey Arena   
2                  Bakery                  Park                   Pub   
3  Furniture / Home Store           Event Space     Accessories Store   
4                   Diner           Yoga Studio              Creperie   
7              Beer Store                   Gym      Asian Restaurant   

   5th Most Common Venue  6th Most Common Venue 7th Most Common Venue  \
1  Portuguese Restaurant          Women's Store   Distribution Center   
2                   Café         Breakfast Spot               Theater   
3               Boutique  Vietnamese Restaurant             Gift Shop   
4               Beer Bar         Sandwich Place          Burger Joint   
7    Japanese Restaurant             Restaurant        Sandwich Place   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
1          Dessert Shop    Dim Sum Restaurant                  Diner  
2            Restaurant    Mexican Restaurant         Chocolate Shop  
3    Miscellaneous Shop           Coffee Shop    Distribution Center  
4         Burrito Place                  Café     College Auditorium  
7   Sporting Goods Shop    Italian Restaurant              Bike Shop

           
Cluster  1  Summary


Borough  Longitude  Cluster Labels 1st Most Common Venue  \
32  Scarborough  -79.239476             1.0     Convenience Store   
35    East York  -79.338106             1.0     Convenience Store   
64         York  -79.518188             1.0                  Park   
66   North York  -79.400049             1.0                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
32            Playground         Women's Store    Dim Sum Restaurant   
35                  Park           Coffee Shop      Doner Restaurant   
64     Convenience Store          Dessert Shop    Dim Sum Restaurant   
66                  Bank                   Bar     Convenience Store   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
32                 Diner        Discount Store   Distribution Center   
35    Dim Sum Restaurant                 Diner        Discount Store   
64                 Diner        Discount Store   Distribution Center   
66         Women's Store      Doner Restaurant                 Diner   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
32               Dog Run      Doner Restaurant             Donut Shop  
35   Distribution Center               Dog Run          Women's Store  
64               Dog Run      Doner Restaurant          Deli / Bodega  
66        Discount Store   Distribution Center                Dog Run

           
Cluster  2  Summary


...

Borough  Longitude  Cluster Labels 1st Most Common Venue  \
16   York  -79.428191             7.0                 Trail   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
16               Dog Run                 Field          Hockey Arena   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
16          Dessert Shop    Dim Sum Restaurant                 Diner   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
16        Discount Store   Distribution Center          Women's Store

           
Cluster  8  Summary


Borough  Longitude  Cluster Labels 1st Most Common Venue  \
94  Etobicoke  -79.594054             8.0   Rental Car Location   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
94             Drugstore         Women's Store               Dog Run   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
94          Dessert Shop    Dim Sum Restaurant                 Diner   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
94        Discount Store   Distribution Center       Doner Restaurant

           
Cluster  9  Summary


Borough  Longitude  Cluster Labels 1st Most Common Venue  \
6  Scarborough  -79.194353             9.0  Fast Food Restaurant   

        2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
6  Construction & Landscaping         Women's Store      Doner Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
6    Dim Sum Restaurant                 Diner        Discount Store   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
6   Distribution Center               Dog Run              Drugstore